In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
from google.colab import files
uploaded = files.upload()

df = pd.read_excel("천재교육_연수원_후기최종.xlsx")

Saving 천재교육_연수원_후기최종.xlsx to 천재교육_연수원_후기최종.xlsx


In [6]:
!pip install transformers
from transformers import pipeline

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [10]:
nlp = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [11]:
nlp("정말 좋아요!")

[{'label': '5 stars', 'score': 0.5692892670631409}]

In [14]:
df

0             파워포인트 주로 사용하는데 매일 하던거만 하다 새로운걸 알아가네요~감사합니다~
1       도형을 이용해 간단하게 손그림 그리는 방법을 배우는 유익한 연수였어요..그런데 프로...
2       저는 미술에 정말 소질이 없는 사람입니다.. 그래서 아이들과 미술을 하는 것도 힘들...
3       교직 생활이 연수가 늘어갈수록 다양한 독서 연수 강좌를 들어왔습니다. 이번 연수는 ...
4       근래 들어 정말 기분좋게, 즐겁게 들었던 연수입니다.환경인가? 하며 들었는데 다시 ...
                              ...                        
2893    학교 진로교육의 목표 (2015개정)학생 자신의 진로를 창의적으로 개발하고 지속적으...
2894    선생님들의 수학 개념 설명을 듣고, 실제 수학 수업을 볼 수 있어서 너무 유익했습니...
2895                             진로라는게 흥미는 있지만 너무 포괄적이네요.
2896                     진로교육에 관해 많은 것을 알게 되었습니다.\n감사합니다.
2897    자녀가 현재 고등학교에 재학중이고 또한 오래전부터 진로교육에 관심이 있었기에 더욱 ...
Name: 후기, Length: 2898, dtype: object

In [13]:
df = df['후기']

In [15]:
train_length = df.astype(str).apply(len)
train_length.head()

0     43
1    143
2    720
3    591
4    462
Name: 후기, dtype: int64

In [16]:
df_list = df.values.tolist()

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

# 데이터를 담고 있는 리스트 형태의 text_list를 가정합니다.
# text_list = [text_1, text_2, ...]

# 사전 학습된 감정 분석을 위한 tokenizer와 model을 불러옵니다.
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 데이터를 토크나이징하고 모델에 입력할 형태로 변환합니다.
tokenized_texts = tokenizer(df_list, padding=True, truncation=True, return_tensors="pt")
input_ids = tokenized_texts["input_ids"]
attention_mask = tokenized_texts["attention_mask"]

# 데이터를 DataLoader로 불러와서 배치 처리합니다.
batch_size = 16
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask)
dataloader = DataLoader(dataset, batch_size=batch_size)

# 결과를 저장할 리스트를 초기화합니다.
results = []

# 배치 단위로 데이터를 모델에 입력하여 감정 분석 결과를 추출합니다.
model.eval()
with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing"):
        batch_input_ids = batch[0]
        batch_attention_mask = batch[1]
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        results.extend(predictions.cpu().numpy())

# 결과를 확인합니다.
print(results)


Processing: 100%|██████████| 182/182 [1:59:29<00:00, 39.39s/it]

[4, 1, 1, 3, 1, 4, 3, 4, 4, 4, 4, 2, 2, 3, 4, 1, 1, 4, 4, 3, 4, 0, 2, 2, 1, 4, 4, 4, 4, 2, 4, 4, 4, 4, 2, 1, 1, 2, 4, 1, 4, 3, 2, 3, 3, 3, 4, 4, 4, 4, 3, 2, 4, 4, 4, 2, 4, 3, 4, 3, 2, 3, 2, 3, 1, 4, 3, 3, 0, 3, 4, 3, 4, 4, 2, 4, 2, 2, 0, 1, 2, 3, 0, 0, 2, 4, 4, 1, 4, 4, 1, 1, 3, 3, 4, 4, 4, 3, 3, 3, 4, 1, 0, 1, 4, 4, 2, 4, 4, 4, 4, 3, 4, 4, 3, 2, 4, 1, 1, 4, 2, 3, 4, 1, 1, 2, 4, 1, 1, 0, 4, 1, 2, 3, 4, 0, 1, 3, 4, 4, 2, 4, 4, 1, 3, 1, 4, 4, 4, 4, 1, 4, 2, 4, 3, 3, 2, 3, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 1, 2, 4, 3, 3, 4, 3, 4, 4, 4, 3, 4, 4, 3, 2, 2, 3, 2, 3, 4, 4, 3, 1, 2, 1, 4, 4, 2, 4, 4, 3, 3, 1, 1, 2, 1, 1, 3, 1, 0, 1, 2, 4, 0, 1, 4, 1, 1, 3, 4, 3, 4, 4, 4, 3, 4, 3, 4, 4, 3, 4, 3, 4, 3, 2, 4, 1, 4, 4, 1, 2, 3, 3, 3, 3, 4, 4, 4, 3, 3, 2, 2, 2, 4, 4, 4, 1, 3, 3, 4, 4, 2, 1, 4, 2, 1, 3, 3, 4, 3, 1, 4, 4, 4, 1, 4, 1, 4, 2, 3, 4, 0, 0, 1, 3, 4, 4, 4, 3, 3, 2, 1, 1, 4, 4, 4, 2, 4, 3, 1, 4, 1, 2, 4, 4, 4, 3, 3, 4, 4, 1, 4, 4, 1, 1, 4, 4, 3, 3, 4, 4, 1, 4, 1, 1, 4, 2, 3, 3, 3, 3, 

In [18]:
results

[4,
 1,
 1,
 3,
 1,
 4,
 3,
 4,
 4,
 4,
 4,
 2,
 2,
 3,
 4,
 1,
 1,
 4,
 4,
 3,
 4,
 0,
 2,
 2,
 1,
 4,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 2,
 1,
 1,
 2,
 4,
 1,
 4,
 3,
 2,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 2,
 4,
 4,
 4,
 2,
 4,
 3,
 4,
 3,
 2,
 3,
 2,
 3,
 1,
 4,
 3,
 3,
 0,
 3,
 4,
 3,
 4,
 4,
 2,
 4,
 2,
 2,
 0,
 1,
 2,
 3,
 0,
 0,
 2,
 4,
 4,
 1,
 4,
 4,
 1,
 1,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 1,
 0,
 1,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 2,
 4,
 1,
 1,
 4,
 2,
 3,
 4,
 1,
 1,
 2,
 4,
 1,
 1,
 0,
 4,
 1,
 2,
 3,
 4,
 0,
 1,
 3,
 4,
 4,
 2,
 4,
 4,
 1,
 3,
 1,
 4,
 4,
 4,
 4,
 1,
 4,
 2,
 4,
 3,
 3,
 2,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 1,
 2,
 4,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 2,
 2,
 3,
 2,
 3,
 4,
 4,
 3,
 1,
 2,
 1,
 4,
 4,
 2,
 4,
 4,
 3,
 3,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 0,
 1,
 2,
 4,
 0,
 1,
 4,
 1,
 1,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 2,
 4,
 1,
 4,
 4,
 1,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 4,


In [20]:
!pip install openpyxl

In [22]:
results = pd.DataFrame({"results":results})

In [25]:
results.to_excel("천재교육_최종_nlptown.xlsx")